In [1]:
import numpy as np
import pdb

import meerkat as mk
from meerkat.contrib.eeg import build_stanford_eeg_dp
from domino.utils import split_dp

## Load EEG datapanel

In [2]:
dp_art = build_stanford_eeg_dp.out(run_id=409)
dp=dp_art.load()
dp.lz[0]

{'file_id': 'DA0551U5_1-1+',
 'filepath': '/media/4tb_hdd/eeg_data/lpch/lpch/DA0551U5_1-1+.eeghdf',
 'fm_split': 'train',
 'id': 'DA0551U5_1-1+_-1.0',
 'sz_start_index': -1.0,
 'target': False,
 'index': '1869',
 'input': LambdaCell(fn=functools.partial(<function stanford_eeg_loader at 0x7f0da6f01280>, clip_len=60)),
 'findings': "Baseline EEG  There is a persistent suppression-burst pattern with typical 20-60s interburst itnervals and 2-10s burst periods.    Between 04:41 and 06:19:57 on 7/30/14 push buttons events occur coinciding exactly with the patient's bursts. On video small should movements can sometimes be seen involving the patient's shoulders. The nurses appear to describe small shoulder shrugs. There are no log entries. These are likely small myoclonic jerks vs stereotyped motor patterns triggered by bursts.    Interpreting Attending: Christopher William Lee-Messer, MD, PhD 23:03 7/29/2014",
 'patient_id': '42132928',
 'age': 2.0454249746321667,
 'duration': 810.99999999999

In [3]:
len(dp)

6254

In [4]:
len(dp.lz[~dp["id"].duplicated()])

6254

## split data

In [17]:
dp_splits_art = split_dp.out(412) #split_dp(dp_art, split_on="patient_id")
dp_splits = dp_splits_art.load()
print(len(dp_splits))
dp_splits.head()

6254


,patient_id (PandasSeriesColumn),split (PandasSeriesColumn),id (PandasSeriesColumn),index (PandasSeriesColumn)
0,42132928,train,DA0551U5_1-1+_-1.0,0
1,46946752,train,FA2730SI_1-1+_-1.0,1
2,43007731,train,CA0555NE_1-1+_-1.0,2
3,45527801,train,DA0551YY_1-1+_-1.0,3
4,45971843,train,FA046044_1-1+_-1.0,4


In [6]:
dp_splits['id'].nunique()

6254

In [7]:
age_labels = dp["age"] > 10
sz_labels = dp["target"]
np.corrcoef([age_labels,sz_labels])[1,0]

-0.014004347407745615

## Slice dp based on metadata

In [8]:
from domino.slices.eeg import build_slice

#dp_train = dp.lz[dp["split"]=="train"]
dp_age_art = build_slice("correlation", dp_art, dp_splits_art, correlate="age", corr=0.5, n=2500, correlate_threshold=10)
dp_age = dp_age_art.load()

task: build_slice, run_id=413


In [9]:
len(dp_age)

2498

In [10]:
age_labels = dp_age["age"] > 10
sz_labels = dp_age["target"]
np.corrcoef([age_labels,sz_labels])[0,1]

0.4993963743737434

In [14]:
(dp_age["split"]=="valid").sum()

294

## Get multiple slices

In [18]:
from domino.slices.eeg import collect_correlation_slices

# correlate_list = ["age"]
# corr_list = [0, 0.3, 0.5, 0.9]
# correlate_thresholds = [10]
# dp_slices_art = collect_correlation_slices(dp_splits_art, correlate_list, corr_list, correlate_thresholds)
dp_slices_art = collect_correlation_slices.out(414)
dp_slices = dp_slices_art.load()
dp_slices.head()

,corr (NumpyArrayColumn),correlate (PandasSeriesColumn),correlation_threshold (NumpyArrayColumn),dp (ListColumn),n (NumpyArrayColumn),slice_category (PandasSeriesColumn),index (PandasSeriesColumn)
0,0.0,age,10,"DataPanel(nrows: 6254, ncols: 4)",2500,correlation,0
1,0.3,age,10,"DataPanel(nrows: 6254, ncols: 4)",2500,correlation,1
2,0.5,age,10,"DataPanel(nrows: 6254, ncols: 4)",2500,correlation,2
3,0.9,age,10,"DataPanel(nrows: 6254, ncols: 4)",2500,correlation,3
